In [8]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

In [416]:
with open(r'C:\Users\konovale\R kansio\METH&BS\new_corpus.csv') as corpus:
    reader = corpus.readlines()
    mydata = list(reader)

In [417]:
mydata

['veljekset Johan Gustav Wilhelm ja Karl Magnus Emil Brummer korottaa aatelisarvo, ja he suku merkitä 28.2.1870 Suomi ritarihuone aatelissuku 112.\n',
 'hän saada kanslianeuvos arvo 1889, valtioneuvos arvo 1893 ja todellinen valtioneuvos arvo 1899.\n',
 'Emma Irene Åström tulla Helsinki yliopisto filosofinen tiedekunta ensimmäinen naispuolinen kunniatohtori 1927.\n',
 '..se ei estää Heikel nimitys ensimmäinen kansatiede dosentti 1889.\n',
 'auskultoida Heinola tuomiokunta hän saada varatuomari arvo 1879.\n',
 'tunnustus elämäntyö Sievers saada 1923, jolloin hän antaa professori arvonimi sekä edelleen 1928, jolloin hän 76vuotias nimittää arkkiatri.\n',
 'Rindell olla näet Mustialaansio tulla jo palkita professori arvonimi 1893, mutta tieteellisesti arvoka julkaisutoiminta hän olla niukalti.\n',
 'tieteellinen julkaisu hän julkaista vähän ja vasta ikämies, mutta hän saada silti professori arvonimi 1907.\n',
 'nämä ansio huomata: Konstantin Aladi nuori saada kauppaneuvos arvonimi 1904.\n'

In [268]:
#new_data = []
#for i in mydata:
#    i = i[:-2]
#    new_data.append(i)

In [418]:
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(mydata)]

In [419]:
#so far the best results for test sentences are with this number of epochs and this vector size: 30 and 20
max_epochs = 30
vec_size = 17
alpha = 0.025

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
model.build_vocab(tagged_data)

In [420]:
for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
    
model.save("d2v.model")
print("Model Saved")

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
Model Saved


In [422]:
model= Doc2Vec.load("d2v.model")
#to find the vector of a document which is not in training data
test_data = word_tokenize("Hän sai nimityksen kenraaliksi".lower())
v1 = model.infer_vector(test_data)
#print("V1_infer", v1)

test_data_M = word_tokenize("Hän avioitui Pietarissa 2. toukokuuta 1892 venäläisen kenraalimajurin varakkaan perijättären Anastasia Arapovan kanssa.".lower())
v_m = model.infer_vector(test_data_M)

test_data_L = word_tokenize("Lönnrot meni naimisiin vuonna 1849 kappalaisen tyttären Maria Piponiuksen kanssa ja lähti opettamaan kirjallisuutta.".lower())
v_l = model.infer_vector(test_data_L)

test_data_N = word_tokenize("Tytär Karin solmi avioliiton ministeri Henrik Ramsayn kanssa ja toimi Suomen Punaisessa Ristissä.".lower())
v_n = model.infer_vector(test_data_N)

# to find most similar doc using tags
#similar_doc = model.docvecs.most_similar('2')
#print(similar_doc)

similar_doc1 = model.docvecs.most_similar([v1])
similar_doc_v_m = model.docvecs.most_similar([v_m],topn=10)
similar_doc_v_l = model.docvecs.most_similar([v_l],topn=10)
similar_doc_v_n = model.docvecs.most_similar([v_n],topn=10)
#print(similar_doc1)
print(similar_doc_v_m)
print(similar_doc_v_l)
print(similar_doc_v_n)

# to find vector of doc in training data using tags or in other words, printing the vector of document at index 1 in training data
#print(model.docvecs['1'])

[('27', 0.6963678002357483), ('22', 0.68153977394104), ('76', 0.6719655394554138), ('25', 0.6616845726966858), ('21', 0.6556556820869446), ('157', 0.6308171153068542), ('198', 0.6302379965782166), ('23', 0.624057948589325), ('189', 0.6208353042602539), ('68', 0.6095075607299805)]
[('23', 0.13377512991428375), ('189', 0.08624720573425293), ('22', 0.08456331491470337), ('145', 0.05946550890803337), ('78', 0.05627712234854698), ('148', 0.03999495506286621), ('154', 0.02445557340979576), ('149', 0.023879773914813995), ('143', 0.017699720337986946), ('52', 0.014712880365550518)]
[('22', 0.6840628385543823), ('192', 0.6476923227310181), ('23', 0.6463348269462585), ('315', 0.480753093957901), ('189', 0.4704722464084625), ('21', 0.46617254614830017), ('266', 0.4640052020549774), ('25', 0.4380340576171875), ('137', 0.43303754925727844), ('20', 0.43024328351020813)]


In [429]:
print(mydata[107])
#27,25,21,6,137,22,192
#23,189,25
#upd after adding the sentence: + 137,  154 (?! negative), 192, 25, 22 (also negative)
from operator import itemgetter
#print(*itemgetter(276,123,234,75,13, 262)(mydata))
#print(*itemgetter(296,6,293,154,380,2)(mydata))
#print(*itemgetter(44,249,86,9,87,253)(mydata))
print(model.docvecs.most_similar([model['naimisiin']]))
print(model.docvecs.most_similar([model['avioitua']]))
print(model.docvecs.most_similar([model['avioliitto']]))

vuosi 1895 hän muuttaa Turku ensin läänisairaala ali-lääkäri ja sitten sama sairaala ylilääkäri 1899, mikä virka hän pysyä eläke jäädä asti 1928.

[('350', 0.7831231355667114), ('23', 0.7823107242584229), ('180', 0.7546542882919312), ('316', 0.7542555332183838), ('189', 0.7463042736053467), ('22', 0.7123032808303833), ('145', 0.6880351305007935), ('299', 0.68767249584198), ('181', 0.6724562048912048), ('155', 0.6524287462234497)]
[('21', 0.8781343698501587), ('23', 0.7849961519241333), ('27', 0.7636508345603943), ('25', 0.7487219572067261), ('22', 0.7202984094619751), ('192', 0.7149435877799988), ('170', 0.6967310309410095), ('76', 0.6512880325317383), ('189', 0.6428390145301819), ('94', 0.6211383938789368)]
[('22', 0.900201141834259), ('23', 0.7920703291893005), ('192', 0.7810046076774597), ('21', 0.7547167539596558), ('27', 0.7032747864723206), ('146', 0.694972813129425), ('14', 0.6924290060997009), ('181', 0.6843852400779724), ('137', 0.6786996126174927), ('315', 0.6674879789352417)

In [309]:
from operator import itemgetter
#print(*itemgetter(276,123,234,75,13, 262)(mydata))
print(*itemgetter(6,211,178,181,177,408)(mydata))

" Naimattomana elänyt Emma Irene Åström poltti Cygnaeuksen kuoltua 1888 omat kirjeensä tälle syistä, joita ei tiedetä. Syviä, rakkaudeksi tulkittavia Emma Irene Åströmin tunteet olivat olleet yksinäistä ja oppinutta Rooman kirjallisuuden professoria Fredrik Julius Peterseniä kohtaan, jonka seurassa viettämiään hetkiä hän muisteli vuosikymmeniä myöhemmin ""ylimaallisen ihanina"""". Petersenin muotokuva koristi hänen kotiaan koko hänen loppuelämänsä ajan. Emma Irene Åströmistä tuli Helsingin yliopiston filosofisen tiedekunnan ensimmäinen naispuolinen kunniatohtori 1927."
 " Ahrenbergin parhaina teoksina on pidetty romaaneja Familjen på Haapakoski (Haapakoskelaiset, 1893) ja Vår landsman (Kansalaisemme, 1897), joissa kuvataan venäläistyneitä suomalaisia upseereita ja heidän perheitään ja jotka nekin osaltaan liittyvät nimenomaan Itä-Suomeen. Niiden saamaan suosioon vaikutti varmaankin myös se, että niissä käytettiin esikuvana Suomen ministerivaltiosihteerin Woldemar von Daehnin perhettä. 

In [ ]:
#the result for similarity with epochs=30 and vector_size=25
print(mydata[4], mydata[176],mydata[237], mydata[174], mydata[138], mydata[162], mydata[161], mydata[169], mydata[94], mydata[112],)

In [195]:
#the first result for similarity
print(mydata[169], mydata[4], mydata[176], mydata[17], mydata[94], mydata[139], mydata[115], mydata[183], mydata[262],mydata[174])

Hän sai molempain oikeuksien kandidaatin arvon toukokuussa 1888 ja ilmoittautui auskultantiksi Vaasan hovioikeuteen.
 Auskultoituaan Heinolan tuomiokunnassa hän sai varatuomarin arvon 1879. 
 Hän valmistui 1896 ylioppilaaksi Nya svenska läroverketistä ja 1899 filosofian maisteriksi pääaineinaan taidehistoria ja nykyiskansain kirjallisuus. 
 Brummer määrättiin 24.12.1884 114. Novotoržokin jalkaväkirykmentin komentajaksi, mistä tehtävästä hän sai eron asetettuna armeijan jalkaväen reserviin 7.2.1886.
 Rafael Hertzberg pääsi ylioppilaaksi 1865, valmistui filosofian kandidaatiksi 1872 ja lisensiaatiksi 1889 väitöskirjallaan Vidskepelsen i Finland på 1600-talet.
 Tuona aikana sairaus sai hänestä uuden otteen, eikä hän enää toipunut. 
 Opintomatkat suuntautuivat 1891–1892 Skandinavian maihin, Saksaan ja Sveitsiin. 
 Valmistuttuaan lääketieteen lisensiaatiksi 1892 hän toimi sukunsa kotikaupungissa Porvoossa yksityislääkärinä vuoden, mutta lähti sitten ensin Mäntyharjulle ja sieltä assistentti